## Cleanly remove all "(USCDI)" tags from short and uscdi extension from StructureDefinitions

1. get each StuctureDefinition in resources-yaml
2. cycle through the elements and look for and remove:
   1.  "(USCDI)" tags in short !! if identical to snapshot then remove the short element altogether
   2.   uscdi extension
3. save each StuctureDefinition in resources-yaml
4. test for errors and issues

In [5]:
from yaml import load as yload, dump as ydump, Loader
from json import loads, dumps
from pathlib import Path
from copy import deepcopy
from datetime import datetime
from fhir.resources.structuredefinition import StructureDefinition
from re import sub

my_path = r'/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input'
# my_path = r'/Users/ehaas/Documents/FHIR/US-Core/input'
in_path = Path(my_path) / 'resources-yaml'
out_path = Path(my_path) / 'resources-yaml-old'
# out_path.mkdir(parents=True, exist_ok=True)
#create out_path using pathlib Path.mkdir(parents=True, exist_ok=True)

R4_path = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.r4.core#4.0.1/package')

today = datetime.today().strftime('%Y-%m-%d')
today, in_path , out_path

('2023-10-17',
 PosixPath('/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml'),
 PosixPath('/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml-old'))

In [6]:
# %%bash -s "$in_path" "$out_path"
# ##### DO THIS ONCE TO preserve the original files copy resources-yaml to resources-yaml-old####
# ##### DON'T RUN THIS AGAIN ####
# my_path='/Users/ehaas/Documents/FHIR/US-Core/input'
# echo $my_path
# cp $1/Struct*.yml $2

In [7]:
%%bash -s "$in_path" "$out_path"
##### To Start Over copy resources-yaml-old to resources-yaml####
cp $2/Struct*.yml $1

In [8]:
def get_shorty(r_type, element_path, shorty):
    for sd_base in R4_path.glob(f'StructureDefinition*-{r_type}.json'):
        try:
            r_base = loads(sd_base.read_text())
        except Exception as e:
            print("Exception: {}".format(type(e).__name__))
            print("Exception message: {}".format(e))
        print(f"======{sd_base}, {r_base['type']}======")
        #fix choice types e.g.fix valueQuantity to value[x] and effectiveDateTime to effective[x]
        element_path = sub(r"value[A-Z].[^\.]*", "value[x]", element_path)
        element_path = sub(r"effective[A-Z].[^\.]*", "effective[x]", element_path)
        for element in r_base['differential']['element']:
            print(f"(element['id'] =  {element['id']}) == (element_path = {element_path})? {element_path == element['id']}❗❗❗")
            if element['id'] == element_path:
                if element['short'] == shorty:
                    return True
                else:
                    return False
        print(f'❗❗❗element not found in base SD - look at the datatype in the path {element_path}')
        #use the penultimate element in element_path to find the datatype in the base SD
        parent_element_path = element_path.rsplit('.', 1)[0]
        #use the last two element in element_path to find the datatype element in the base SD
        datatype_element_path = element_path.split('.', )[-1]

        print(f"parent_element_path = {parent_element_path} datatype_element_path = {datatype_element_path}")

        #get the datatype
        for element in r_base['differential']['element']:
            if element['id'] == parent_element_path:
                for type in element['type']:
                    print(f"datatype = {type['code']}")
                    #get the datatype SD short and compare to the shorty
                    path = f"{type['code']}.{datatype_element_path}"
                    print(f"type = {type['code']}, path = {path}, shorty = {shorty}")
                    if get_shorty(type['code'], (f"{type['code']}.{datatype_element_path}"), shorty):
                        return True  # !!!doesn't not account for if element not found in datatype!!!

                    



               

#todo make a deepcopy and modify that copy instead of the original

for sd in in_path.glob('StructureDefinition*.yml'):
    print(f"***********{sd}***********")
    r = yload(sd.read_text(), Loader=Loader)
    r_copy = deepcopy(r)
    print(f"☞ ☞ ☞ Type = {r['type']}")
    for i, element in enumerate(r['differential']['element']):
        print(f"☞ ☞ ☞ element {i}: {element['id']}")

        try:
            print(f"\U00002753 \U00002753 \U00002753 more than 1 extension? = {len(element['extension']) > 1} ❗❗❗")
            for j, extension in enumerate(element['extension']):
                if extension['url'] == 'http://hl7.org/fhir/us/core/StructureDefinition/uscdi-requirement':
                    if len(element['extension']) == 1:

                        print('❗❗❗remove the extension element in the copy')
                        r_copy['differential']['element'][i].pop('extension')
                        r_copy['date'] = today
                    else:
                        print('❗❗❗remove the extension in the copy')
                        r_copy['differential']['element'][i].extension.pop(j)
                        r_copy['date'] = today
                # print(extension['url'], extension['valueBoolean'])
            modified_short = element['short'].replace('(USCDI) ','').replace('(ADD\'L USCDI) ','').replace('(USCDI Requirement) ','').replace("❗(𝗔𝗗𝗗'𝗟 𝗨𝗦𝗖𝗗𝗜) ","")
            print(f" 👀 modify {element['short']} to {modified_short}")
            if modified_short != element['short']:
                print('❗❗❗modify the short in the copy')
                r_copy['differential']['element'][i]['short'] = modified_short
                r_copy['date'] = today
            # compare short to the R4 fhir element
            shorty = get_shorty(r['type'], element['path'], modified_short)
            print(f"\U00002753 \U00002753 \U00002753 Is the short '{modified_short}' the same as the base - {shorty} ❗❗❗")
            if shorty: 
                print('❗❗❗remove short from the copy')
                r_copy['differential']['element'][i].pop('short')


            print('❗❗❗ done with element')
            print('-'*10)

        except Exception as e:
            print(f"Exception: {repr(e)}")
            print("❗❗❗ if exception is KeyError('extension') then no extension on element")
            print('-'*10)
    print('❗❗❗ done with SD')
    print('-'*40)
    print('\n')

    # custom sort the keys in the elements using fhir.resources
    r_copy = StructureDefinition.parse_obj(r_copy)

    # save the copy to resources-yaml using Pathlib
    copyfile = in_path / sd.name
    copyfile.write_text(r_copy.yaml(indent=2))


***********/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml/StructureDefinition-us-core-sex.yml***********
☞ ☞ ☞ Type = Extension
☞ ☞ ☞ element 0: Extension
Exception: KeyError('extension')
❗❗❗ if exception is KeyError('extension') then no extension on element
----------
☞ ☞ ☞ element 1: Extension.url
Exception: KeyError('extension')
❗❗❗ if exception is KeyError('extension') then no extension on element
----------
☞ ☞ ☞ element 2: Extension.value[x]
Exception: KeyError('extension')
❗❗❗ if exception is KeyError('extension') then no extension on element
----------
❗❗❗ done with SD
----------------------------------------


***********/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml/StructureDefinition-us-core-specimen.yml***********
☞ ☞ ☞ Type = Specimen
☞ ☞ ☞ element 0: Specimen
Exception: KeyError('extension')
❗❗❗ if exception is KeyError('extension') then no extension on element
----------
☞ ☞ ☞ element 1: Specimen.identifier
❓ ❓ ❓ more than 1 extension? = False ❗❗❗
❗❗